# Model Name TBD
Combines the time-distributed feature extraction of Convolutional LSTMs with the upsampling and skip connections of a U-Net to convert video-like input features and time-distributed vector metadata into a next frame semantic segmentation map.

### Import Block

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.utils import class_weight

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.initializers import Constant
#from skimage.metrics import structural_similarity as ssim
#from skimage.metrics import mean_squared_error
#from math import log10, sqrt

#%matplotlib inline

### Function Definitions

In [2]:
# Loader functions
# Inspiration: https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a

def get_2d_input(path):
    # Load array.
    t_2d_input = np.load(path)
    
    #return t_2d_input
    return t_2d_input[:,:,:,:1]

def get_1d_input(path):
    # Load array.
    t_1d_input = np.load(path)
    
    # Expand dimensions to match model input.
    t_1d_input = tf.expand_dims(tf.expand_dims(t_1d_input, 2), 2)
    
    # Put channel dim at the end.
    t_1d_input = np.moveaxis(t_1d_input, 1, -1)
    
    return t_1d_input

def get_output(path):
    # Load array.
    t_output = np.load(path)
    
    # Put channel dim at the end.
    t_output = np.moveaxis(t_output, 0, -1)
    return t_output

def data_generator(samples, num_samples, batch_size = 64, calculated_sample_weights = None):
    
    while True:
        # Suffle data at the start of each epoch.
        sample_indicies = np.arange(num_samples)
        np.random.shuffle(sample_indicies)
        n = 0
        
        while n + batch_size < num_samples:
            # Get indicies for the batch
            batch_samples  = sample_indicies[n:n + batch_size]
            n += batch_size

            batch_input_2d  = []
            batch_input_1d  = []
            batch_output = [] 
            batch_sample_weights = []

            # Read in each input, perform preprocessing and get labels
            for sample in batch_samples:
                input_2d = get_2d_input(samples.iloc[sample].features_2d)
                input_1d = get_1d_input(samples.iloc[sample].features_1d)
                output = get_output(samples.iloc[sample].labels)
                
                batch_input_2d += [input_2d]
                batch_input_1d += [input_1d]
                batch_output += [output]

                if type(calculated_sample_weights) != type(None):
                    sample_weights = calculated_sample_weights[sample]
                    batch_sample_weights += [sample_weights]
                
            # Return a tuple to feed the network
            batch_x = np.array(batch_input_2d)
            batch_v = np.array(batch_input_1d)
            batch_y = np.array(batch_output)
            
            if type(calculated_sample_weights) == type(None):
                yield(batch_x, batch_y)
            else:
                batch_sample_weights = np.array(batch_sample_weights)
                yield(batch_x, batch_y, batch_sample_weights)

In [3]:
# Solution for problem with class_weights not working with 3D outputs in tensorflow.
# From: https://github.com/keras-team/keras/issues/3653
def generate_sample_weights(training_data, class_weights): 
    #replaces values for up to 3 classes with the values from class_weights#
    sample_weights = [np.where(y==0,class_weights[0],
                        np.where(y==1,class_weights[1],
                        y)) for y in training_data]
    return np.asarray(sample_weights)

In [4]:
# SSIM/PSNR loss functions.
# Inspiration: https://stackoverflow.com/questions/57357146/use-ssim-loss-function-with-keras
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

def psnr_loss(y_true, y_pred):
    return (100 - tf.reduce_mean(tf.image.psnr(y_true, y_pred, 1.0))) / 100

### Model Assembly

In [5]:
## Hyperparameters
top_features = 32
condensed_features = 32
upsample_filters = 32
fc_filters = 32

In [6]:
# Inputs broken out by array and vector features.
inputs_2d = layers.Input(shape=((10,32,32,1)))
#inputs_1d = layers.Input(shape=((10,1,1,192)))

In [7]:
x = layers.ConvLSTM2D(
    filters=128,
    kernel_size=(5, 5),
    padding='same',
    return_sequences=True,
)(inputs_2d)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding='same',
    return_sequences=True,
)(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding='same',
    return_sequences=False,
)(x)
outputs = layers.Conv2D(1, 1, padding='same', activation = 'sigmoid', name = 'outputs')(x)
outputs

<KerasTensor: shape=(None, 32, 32, 1) dtype=float32 (created by layer 'outputs')>

In [8]:
combo_model = tf.keras.Model(inputs = inputs_2d, outputs = outputs, name = 'micro_model')

In [9]:
combo_model.summary()

Model: "micro_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 32, 32, 1)]   0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 10, 32, 32, 128)   1651712   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 10, 32, 32, 64)    1229056   
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 32, 32, 64)        819456    
                                                                 
 outputs (Conv2D)            (None, 32, 32, 1)         65        
                                                                 
Total params: 3,700,289
Trainable params: 3,700,289
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile model.
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
#loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = tf.keras.losses.MeanSquaredError()
combo_model.compile(loss=loss_fn, 
                    optimizer=opt, 
                    metrics=[tf.keras.metrics.MeanSquaredError(name='MSE'),
                             tf.keras.metrics.AUC(name='AUC'),
                             ssim_loss,
                             psnr_loss
                            ])

In [11]:
# Test on dummy data to see that shapes look right.
img_batch = tf.zeros([4,10,32,32,1], dtype = 'float32')
#vector_batch = tf.zeros([4,10,1,1,192], dtype = 'float32')
combo_model.predict(img_batch).shape

1/1 [==============================] - 2s 2s/step


(4, 32, 32, 1)

### Data Preparation

In [12]:
# Load metadata on yearly datasets.
df_2017 = pd.read_csv('4fold_super/2017/meta.csv')
df_2018 = pd.read_csv('4fold_super/2018/meta.csv')
df_2019 = pd.read_csv('4fold_super/2019/meta.csv')
df_2020 = pd.read_csv('4fold_super/2020/meta.csv')

# Combine into desired train/val split.
meta_t = pd.concat([df_2017,df_2018,df_2019]).reset_index()
meta_v = df_2020

In [13]:
# Load all labels from the training set into memory to get weights
y_train = []

# Iterate over dataset.
for x in range(0,len(meta_t)):
    y_train.append(np.load(meta_t.iloc[x].labels))

y_train = np.stack(y_train)
y_train = np.minimum(y_train,1)
y_train = tf.expand_dims(y_train, axis = -1).numpy()

# Get class weights for WBCE/MSE.
weights = class_weight.compute_class_weight('balanced',
                                            classes = [0,1],
                                            y = y_train.flatten())
# Examine weights.
weights_dict = {0:weights[0], 1:weights[1]}
weights_dict

{0: 0.5049593185573996, 1: 50.910151537247934}

In [14]:
# Experiment with reducing weights arbitrarily.
#weights_dict = {0:0.6, 1:45.0}

In [15]:
# Get weights.
calculated_sample_weights = generate_sample_weights(y_train, weights)

# Drop y_train to save memory.
y_train = None

In [16]:
# Data loaders.
batch_size = 32
t_gen = data_generator(meta_t, len(meta_t), batch_size = batch_size, calculated_sample_weights = calculated_sample_weights[:,0])
v_gen = data_generator(meta_v, len(meta_v), batch_size = batch_size)

In [17]:
len(next(t_gen)), len(next(v_gen))

(3, 2)

In [18]:
next(v_gen)[0][0].shape

(10, 32, 32, 1)

### Model Training

In [19]:
# Adding callbacks.
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1)

# Train model.
combo_model.fit(t_gen, 
                   epochs = 10, 
                   verbose = 1, 
                   batch_size = batch_size,
                   validation_data = v_gen,
                   #callbacks = [early_stopping, reduce_lr],
                   steps_per_epoch = len(meta_t) // batch_size,
                   validation_steps = len(meta_v) // batch_size
                  )

Epoch 1/10
468/468 [==============================] - 376s 794ms/step - loss: 0.2740 - MSE: 0.1953 - AUC: 0.8554 - ssim_loss: 0.9957 - psnr_loss: 0.9198 - val_loss: 0.1780 - val_MSE: 0.1780 - val_AUC: 0.8975 - val_ssim_loss: 0.9952 - val_psnr_loss: 0.9167
Epoch 2/10
468/468 [==============================] - 377s 807ms/step - loss: 0.2403 - MSE: 0.1506 - AUC: 0.8882 - ssim_loss: 0.9949 - psnr_loss: 0.9066 - val_loss: 0.1659 - val_MSE: 0.1659 - val_AUC: 0.9014 - val_ssim_loss: 0.9940 - val_psnr_loss: 0.9098
Epoch 3/10
468/468 [==============================] - 389s 831ms/step - loss: 0.2371 - MSE: 0.1466 - AUC: 0.8936 - ssim_loss: 0.9944 - psnr_loss: 0.9045 - val_loss: 0.1455 - val_MSE: 0.1455 - val_AUC: 0.9038 - val_ssim_loss: 0.9928 - val_psnr_loss: 0.9020
Epoch 4/10
468/468 [==============================] - 383s 819ms/step - loss: 0.2337 - MSE: 0.1437 - AUC: 0.8970 - ssim_loss: 0.9942 - psnr_loss: 0.9025 - val_loss: 0.1496 - val_MSE: 0.1496 - val_AUC: 0.9054 - val_ssim_loss: 0.9932 

In [25]:
combo_model.save('Models/CM_micro1_ep10_2020')

INFO:tensorflow:Assets written to: Models/CM_micro1_ep10_2020\assets


INFO:tensorflow:Assets written to: Models/CM_micro1_ep10_2020\assets


In [40]:
combo_model = tf.keras.models.load_model('Models/CM_micro1_ep10_2020'
                                         , custom_objects = {'ssim_loss': ssim_loss, 'psnr_loss': psnr_loss})

In [41]:
# Change learning rate.
print('Old LR:',tf.keras.backend.eval(combo_model.optimizer.lr))
tf.keras.backend.set_value(combo_model.optimizer.lr, 0.00001)
print('New LR:',tf.keras.backend.eval(combo_model.optimizer.lr))

# Reload generators.
t_gen = data_generator(meta_t, len(meta_t), batch_size = batch_size, calculated_sample_weights = calculated_sample_weights[:,0])
v_gen = data_generator(meta_v, len(meta_v), batch_size = batch_size)

# Train more.
combo_model.fit(t_gen, 
                   epochs = 10, 
                   verbose = 1, 
                   batch_size = batch_size,
                   validation_data = v_gen,
                   steps_per_epoch = len(meta_t) // batch_size,
                   validation_steps = len(meta_v) // batch_size
                  )

Old LR: 1e-04
New LR: 1e-05
Epoch 1/10
468/468 [==============================] - 380s 803ms/step - loss: 0.2201 - MSE: 0.1300 - AUC: 0.9105 - ssim_loss: 0.9923 - psnr_loss: 0.8952 - val_loss: 0.1461 - val_MSE: 0.1461 - val_AUC: 0.9114 - val_ssim_loss: 0.9921 - val_psnr_loss: 0.9020
Epoch 2/10
468/468 [==============================] - 371s 793ms/step - loss: 0.2195 - MSE: 0.1301 - AUC: 0.9111 - ssim_loss: 0.9925 - psnr_loss: 0.8957 - val_loss: 0.1446 - val_MSE: 0.1446 - val_AUC: 0.9122 - val_ssim_loss: 0.9919 - val_psnr_loss: 0.9013
Epoch 3/10
468/468 [==============================] - 371s 793ms/step - loss: 0.2189 - MSE: 0.1294 - AUC: 0.9116 - ssim_loss: 0.9923 - psnr_loss: 0.8952 - val_loss: 0.1466 - val_MSE: 0.1466 - val_AUC: 0.9106 - val_ssim_loss: 0.9920 - val_psnr_loss: 0.9019
Epoch 4/10
468/468 [==============================] - 371s 793ms/step - loss: 0.2189 - MSE: 0.1289 - AUC: 0.9119 - ssim_loss: 0.9922 - psnr_loss: 0.8949 - val_loss: 0.1489 - val_MSE: 0.1489 - val_AUC: 0.9

In [43]:
combo_model.save('Models/CM_micro1_ep20_2020')

INFO:tensorflow:Assets written to: Models/CM_micro1_ep20_2020\assets


INFO:tensorflow:Assets written to: Models/CM_micro1_ep20_2020\assets


In [42]:
combo_model = tf.keras.models.load_model('Models/CM_micro1_ep20_2020'
                                         , custom_objects = {'ssim_loss': ssim_loss, 'psnr_loss': psnr_loss})

OSError: No file or directory found at Models/CM_micro1_ep20_2020

### Model Evaluation

In [22]:
# Get entire validation set.
x_val, v_val, y_val = [],[],[]
for x in range(0,len(meta_v)):
    x_val.append(np.load(meta_v.iloc[x].features_2d))
    v_val.append(np.load(meta_v.iloc[x].features_1d))
    y_val.append(np.load(meta_v.iloc[x].labels))
    
x_val = np.stack(x_val)
v_val = np.stack(v_val)
y_val = np.stack(y_val)

# Dimension wrangling.
v_val = tf.expand_dims(tf.expand_dims(v_val, 2), 2)
y_val = np.moveaxis(y_val, 1, -1)

In [23]:
x_val[:,:,:,:,:1].shape, next(t_gen)[0][0].shape

((5000, 10, 32, 32, 1), (10, 32, 32, 1))

In [44]:
# Predict on all samples.
all_preds = combo_model.predict(x_val[:,:,:,:,:1])

157/157 [==============================] - 34s 216ms/step


In [45]:
# Compute and show set scores.
set_ssim = tf.image.ssim(tf.cast(y_val, dtype='float32'), all_preds, 1.0)
set_psnr = tf.image.psnr(tf.cast(y_val, dtype='float32'), all_preds, 1.0)
set_mse = tf.keras.metrics.mean_squared_error(y_val, all_preds)
print('Model Prediction Report')
print('SSIM:', np.mean(set_ssim))
print('PSNR:', np.mean(set_psnr) / 100)
print('MSE:', np.mean(set_mse))

Model Prediction Report
SSIM: 0.008846255
PSNR: 0.09997410774230957
MSE: 0.14208356
